Developing a baseline for the fucking task. In this notebook, we are going to train a simple Resnet network to label each patch separately and the final decision makes by a majority voting.

In [5]:
import numpy as np
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim

import os
os.environ["CUDA_VISIBLE_DEVICES"] = "5"
import torch
# from sklearn.model_selection import train_test_split
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader
import pandas as pd
import glob
import numpy as np
import copy

## Functions

In [16]:
class LymphomaDataset(Dataset):
    """
    Dataset class for Lymphoma dataset
    http://andrewjanowczyk.com/use-case-7-lymphoma-sub-type-classification/
    """
    def __init__(self, patchpath_list):#, patch_size=36, step_size=32):
        super(LymphomaDataset, self)
        self.patchpath_list = patchpath_list
#         self.patch_size = patch_size
#         self.step_size  = step_size
        
        file_ids = [fp.split('/')[-1].split('.')[0] for fp in patchpath_list]
        file_names = ['-'.join(fi.split('-')[1:]) for fi in file_ids]
        file_classes = [fi.split('-')[0] for fi in file_ids]
        
        class_names = list(set(file_classes))
        class_names = sorted(class_names)
        class_labels = dict((name, i) for i, name in enumerate(class_names))
        
        self.class_labels = class_labels
        self.file_names = file_names
        self.X_path = patchpath_list
        self.y = [class_labels[cn] for cn in file_classes]
        
        
    def __getitem__(self, idx):
        
        x = imread(self.X_path[idx])
        y = self.y[idx]
        filename = self.file_names[idx]
        classname = [k for k,v in self.class_labels if v==y][0]
        
        x = torch.tensor(x).permute(2,0,1)
        y = torch.nn.functional.one_hot(torch.tensor(y), num_classes = len(self.class_labels))
        
        return {
            'x': x,
            'y': y,
            'filename': filename,
            'classname': classname
        }
    
    def __len__(self):
        return len(self.patchpath_list)

### Model

In [ ]:
class BaselineModel():
    pass

## Global parameters

In [20]:
config = {
    'batch_size_tr'    : 8,
    'batch_size_val'   : 8,
    'validation_split' : 0.2,
    'pathces_dir'      : './patches/',
    'test_size'        : 0.2,
    'saved_model_path' : '',
    'pretrained'       : False
}

In [21]:
patches_fp = glob.glob(f"{config['pathces_dir']}*.tif")
patches_fp.sort()
psfp_train, psfp_val = train_test_split(patches_fp, test_size=config['validation_split'], shuffle=True)
train_dataset = LymphomaDataset(psfp_train)
train_loader  = DataLoader(train_dataset, batch_size=int(config['batch_size_tr']), shuffle=True)
val_dataset   = LymphomaDataset(psfp_val)
val_loader    = DataLoader(val_dataset, batch_size=int(config['batch_size_va']), shuffle=True)

NameError: name 'train_test_split' is not defined

In [ ]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model = models.resnet50(pretrained=True)
for param in model.parameters():
    param.requires_grad = False
    
model.fc = nn.Sequential(nn.Linear(2048, 512),
                                 nn.ReLU(),
                                 nn.Dropout(0.2),
                                 nn.Linear(512, 10),
                                 nn.LogSoftmax(dim=1))
criterion = nn.NLLLoss()
optimizer = optim.Adam(model.fc.parameters(), lr=0.003)
# optimizer = optim.RMSprop(Net.parameters(), lr= float(config['lr']), weight_decay=1e-8, momentum=0.9)

scheduler = optim.lr_scheduler.ReduceLROnPlateau(optimizer, 'min')
model.to(device)
if int(pretrained=True):
    model.load_state_dict(torch.load(saved_model_path, map_location='cpu')['model_weights'])

## Train

In [ ]:
epochs = 1
steps = 0
running_loss = 0
print_every = 10
train_losses, test_losses = [], []
for epoch in range(epochs):
    for batch in train_loader:
        inputs = batch['x']
        labels = batch['y']
        steps += 1
        inputs, labels = inputs.to(device), labels.to(device)
        optimizer.zero_grad()
        logps = model.forward(inputs)
        loss = criterion(logps, labels)
        loss.backward()
        optimizer.step()
        running_loss += loss.item()
        
        if steps % print_every == 0:
            test_loss = 0
            accuracy = 0
            model.eval()
            with torch.no_grad():
                for batch in test_loader:
                    inputs = batch['x']
                    labels = batch['y']
                    inputs, labels = inputs.to(device), labels.to(device)
                    logps = model.forward(inputs)
                    batch_loss = criterion(logps, labels)
                    test_loss += batch_loss.item()
                    
                    ps = torch.exp(logps)
                    top_p, top_class = ps.topk(1, dim=1)
                    equals = top_class == labels.view(*top_class.shape)
                    accuracy += torch.mean(equals.type(torch.FloatTensor)).item()
            train_losses.append(running_loss/len(trainloader))
            test_losses.append(test_loss/len(testloader))                    
            print(f"Epoch {epoch+1}/{epochs}.. "
                  f"Train loss: {running_loss/print_every:.3f}.. "
                  f"Test loss: {test_loss/len(testloader):.3f}.. "
                  f"Test accuracy: {accuracy/len(testloader):.3f}")
            running_loss = 0
            model.train()
torch.save(model, 'basemodel.pth')

## Test